In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchsummary import summary

In [19]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [20]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform = transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform = transform)

In [22]:
batch_size = 128
num_class = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [30]:
model = models.inception_v3(pretrained=True, aux_logits=True) # aux_logits=True is required for pretrained Inception V3

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_class)

# Note: Since aux_logits is now True, the model forward pass will return
# a tuple of (output, aux_output) during training if aux_logits is True.
# However, during evaluation (model.eval()), it will only return the main output.
# The provided training and evaluation loops handle this correctly because
# when aux_logits is True, model(input) will return a tuple during training,
# but the current code only uses the first element of the tuple implicitly
# when calculating the loss and predictions because cross_entropy expects
# the output and label directly.
# If you were to explicitly use the aux_output, you would need to handle the
# tuple output during training, e.g., output, aux_output = model(input)
# and calculate a combined loss if desired.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
print(model.fc)
print(num_ftrs)

Linear(in_features=2048, out_features=10, bias=True)
2048


In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
summary(model, (3,299, 299))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [35]:
# Optimizer and Loss function
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)  # Only train the classifier layer

citerion = nn.CrossEntropyLoss()

def train_model(model, train_loader, epochs):
  model.train()
  for epoch in range(epochs):
    total_running_loss = 0
    correct = 0
    for input, label in train_loader:
      input, label = input.to(device), label.to(device)
      optimizer.zero_grad()
      output = model(input)
      # When aux_logits=True, the model returns a tuple (main_output, aux_output) during training.
      # We need to use the main output for the loss calculation.
      if isinstance(output, tuple):
          output = output[0] # Extract the main output

      loss = citerion(output, label)
      loss.backward()
      optimizer.step()
      total_running_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = total_running_loss / len(train_loader.dataset)
    accuracy = correct / len(train_loader.dataset)
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')


train_model(model, train_loader, epochs)

Epoch: [1/5], Loss: 1.0812, Accuracy: 0.6539
Epoch: [2/5], Loss: 0.8543, Accuracy: 0.7097
Epoch: [3/5], Loss: 0.8324, Accuracy: 0.7132
Epoch: [4/5], Loss: 0.8202, Accuracy: 0.7174
Epoch: [5/5], Loss: 0.8150, Accuracy: 0.7214


In [36]:
def evaluate_model (model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input, label in test_loader:
      input, label = input.to(device), label.to(device)
      output = model(input)
      loss = citerion(output, label)
      test_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = test_loss / len(test_loader.dataset)
    accuracy  = correct / len(test_loader.dataset)
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

evaluate_model(model, test_loader)

Test Loss: 0.6835, Accuracy: 0.7651
